In [294]:
import pandas as pd

In [295]:
club_df_dict = pd.read_excel('Club stats.xlsx',sheet_name=None, engine='openpyxl')

In [296]:
combined_df = pd.DataFrame()
for k,df in club_df_dict.items():
    if k=='Glossary':
        continue
    df['period'] = k
    combined_df = combined_df.append(df,ignore_index=True)

In [297]:
combined_df =combined_df.replace({'Charlotte Bobcats': 'Charlotte Hornets'})

In [298]:
club_df_dict.keys()

dict_keys(['Glossary', '20-21 regular', '19-20 regular', '18-19 regular', '17-18 regular', '16-17 regular', '15-16 regular', '14-15 regular', '13-14 regular', '20-21 post', '19-20 post', '18-19 post', '17-18 post', '16-17 post', '15-16 post', '14-15 post', '13-14 post'])

In [299]:
combined_df[['year','season']] = combined_df.period.str.split(' ',expand=True)

In [300]:
post_yr_list = {
    yr:combined_df[(combined_df.year==yr)&(combined_df.season=='post')].TEAM.unique().tolist() 
    for yr in combined_df.year.unique().tolist()
}
    

In [301]:
reg_df = combined_df[combined_df.season=='regular']
reg_df['playoff'] = combined_df[combined_df.season=='regular'].apply(lambda x: x.TEAM in post_yr_list[x.year],axis=1)

C:\Users\yemao\.pyenv\pyenv-win\versions\3.7.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [302]:
reg_df = reg_df.drop(['period','season','RK'],axis=1)

In [303]:
reg_df = reg_df.sort_values(by=['TEAM','year'])

In [304]:
reg_df['playoff_lag']=reg_df.groupby(['TEAM'])['playoff'].shift(1)

In [305]:
playoff_2012 = pd.read_excel('playoff_2012.xlsx',engine='openpyxl')

In [306]:
playoff_2012

,Rk,Tm
0,1,San Antonio Spurs
1,2,Oklahoma City Thunder
2,3,Miami Heat
3,4,Denver Nuggets
4,5,Los Angeles Lakers
5,6,Dallas Mavericks
6,7,Memphis Grizzlies
7,8,Indiana Pacers
8,9,Los Angeles Clippers
9,10,Boston Celtics


In [307]:
reg_df['lasy_year_playoff']=reg_df.apply(lambda r: True if (r.TEAM in playoff_2012.Tm.tolist()) and (r.year=='13-14') else r.playoff_lag,axis=1)

In [308]:
reg_df.lasy_year_playoff = reg_df.lasy_year_playoff.fillna(False)

In [309]:
reg_df = reg_df.drop('playoff_lag',axis=1).reset_index(drop=True)

In [310]:
reg_df.year.unique().tolist()

['13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20', '20-21']

In [311]:
locations_data = pd.read_csv('NBA clubs Location and Economy.csv')

In [312]:
locations_data = locations_data[['Club name','Year','Metro GDP in $Millon','Metro Population in thousands']]
locations_data.columns = ['TEAM','year','GDP','pop']

In [313]:
year_dict ={2014:'13-14', 2015:'14-15', 2016:'15-16', 2017:'16-17', 2018:'17-18', 2019:'18-19', 2020:'19-20', 2021:'20-21'}

In [314]:
locations_data['year'] = locations_data.year.apply(lambda x: year_dict[x])

In [315]:
reg_df = reg_df.replace({'LA Clippers':'Los Angeles Clippers'})

In [316]:
team_df = reg_df.merge(locations_data,on=['year','TEAM'])

## Processing Player's Data

In [317]:
year_list = list(year_dict.values())

In [318]:
year_list

['13-14', '14-15', '15-16', '16-17', '17-18', '18-19', '19-20', '20-21']

In [319]:
players_df = pd.DataFrame()
for yr in year_list:
    yr_df = pd.read_csv(f'Players_Stats_salary/{yr}'+r' combined.csv')
    yr_df['year'] = yr
    players_df = players_df.append(yr_df)

In [320]:
players_df.columns

Index(['RK_x', 'Player Name', 'Club1', 'Club2', 'Club3', 'POS_x', 'GP', 'MIN',
       'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%',
       'REB', 'AST', 'STL', 'BLK', 'TO', 'DD2', 'TD3', 'PER', 'RK_y', 'NAME',
       'POS_y', 'TEAM', 'SALARY', 'year'],
      dtype='object')

In [321]:
players_df = players_df.drop(['RK_x','Player Name','POS_y'],axis=1)

In [322]:
players_df = players_df[~players_df.SALARY.isna()]

In [323]:
players_df.SALARY = pd.to_numeric(players_df.SALARY.apply(lambda x: str(x).replace('$','').replace(',','')))

In [324]:
top_10_paid = players_df.sort_values(['TEAM','year','SALARY'],ascending=False).groupby(['TEAM','year']).head(10)

In [325]:
top_10_salary_sum_by_team = top_10_paid.groupby(['TEAM','year']).SALARY.sum().reset_index()

In [326]:
top_10_salary_sum_by_team = top_10_salary_sum_by_team.replace({'LA Clippers':'Los Angeles Clippers'})

In [327]:
final_df = team_df.merge(top_10_salary_sum_by_team,on=['TEAM','year'])

In [328]:
final_df.TEAM.unique()

array(['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets',
       'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers',
       'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons',
       'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers',
       'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies',
       'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves',
       'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder',
       'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns',
       'Portland Trail Blazers', 'Sacramento Kings', 'San Antonio Spurs',
       'Toronto Raptors', 'Utah Jazz', 'Washington Wizards'], dtype=object)

In [329]:
players_df_dict = pd.read_excel('Players stats.xlsm',sheet_name=None,engine='openpyxl')

In [330]:
combined_df_players = pd.DataFrame()
for k,df in players_df_dict.items():
    if k=='Glossary':
        continue
    df['period'] = k
    combined_df_players = combined_df_players.append(df,ignore_index=True)

In [331]:
combined_df_players[['year','season']] = combined_df_players.period.str.split(' ',expand=True)

In [332]:
combined_df_players= combined_df_players.drop(['Player Name','Unnamed: 2','period'],axis=1)

In [333]:
players_df = combined_df_players[(combined_df_players.season=='regular')&(~combined_df_players.NAME.isna())].reset_index(drop=True)

In [334]:
players_df = players_df.drop('Clubs',axis=1)

In [335]:
players_df['Club'] = players_df.apply(lambda x: [x.Club1, x.Club2, x.Club3],axis=1)

In [336]:
players_df = players_df.explode('Club').drop(['Club1','Club2','Club3'],axis=1)

In [337]:
players_df = players_df.dropna()

In [338]:
abb = pd.read_excel('Team_Abb.xlsx',engine='openpyxl')

In [339]:
club_abb_corr = {
'GS':'GSW','WSH':'WAS','NO':'NOP','UTAH':'UTA','SA':'SAS','ISAC':'SAS','IORL':'ORL','IIGS':'GSW','IPHI':'PHI',
    'VSA':'SAS','INO':'NOP','NY':'NYK'
}

In [340]:
players_df = players_df.replace(club_abb_corr)

In [341]:
players_df=players_df.merge(abb,left_on='Club',right_on='Abb',how='left')

In [342]:
players_df.columns

Index(['RK', 'NAME', 'POS', 'GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM',
       '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'REB', 'AST', 'STL', 'BLK', 'TO',
       'DD2', 'TD3', 'PER', 'year', 'season', 'Club', 'Abb', 'TEAM'],
      dtype='object')

## Avg Turnover Metrics for 8 most-played players for each team and each year

In [343]:
players_df['time_played'] = players_df.MIN*players_df.GP

In [344]:
top_8_most_played_players = players_df.sort_values(['TEAM','year','time_played'],ascending=False).groupby(['TEAM','year']).head(8)

In [345]:
top_8_most_played_players['turnover_metrics'] = top_8_most_played_players.TO/top_8_most_played_players.FGA

In [346]:
avg_turnover_metrics = top_8_most_played_players.groupby(['TEAM','year']).mean()['turnover_metrics'].reset_index()

In [347]:
final_df = final_df.merge(avg_turnover_metrics,on=['TEAM','year'])

## Assist Metrics

In [348]:
median_assit_for_top_assistor = players_df.sort_values(['TEAM','year','AST'],ascending=False).groupby(['TEAM','year']).head(8)\
.groupby(['TEAM','year']).median()['AST'].reset_index()

In [349]:
median_assit_for_top_assistor = median_assit_for_top_assistor.rename({'AST':'median_ast'},axis=1)

In [350]:
final_df = final_df.merge(median_assit_for_top_assistor,on=['TEAM','year'])

In [351]:
final_df

,TEAM,GP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,...,TO,PF,year,playoff,lasy_year_playoff,GDP,pop,SALARY,turnover_metrics,median_ast
0,Atlanta Hawks,82.0,101.0,37.3,81.6,45.8,9.4,25.8,36.3,17.0,...,14.5,19.2,13-14,True,True,325003,5595,52150108.0,0.168249,3.00
1,Atlanta Hawks,82.0,102.5,38.1,81.7,46.6,10.0,26.2,38.0,16.5,...,13.5,17.8,14-15,True,True,347745,5689,52112642.0,0.168001,2.95
2,Atlanta Hawks,82.0,102.8,38.6,84.4,45.8,9.9,28.4,35.0,15.6,...,14.5,19.1,15-16,True,True,369996,5792,68126061.0,0.172936,3.25
3,Atlanta Hawks,82.0,103.2,38.1,84.4,45.1,8.9,26.1,34.1,18.1,...,15.2,18.2,16-17,True,True,389472,5877,88612062.0,0.167276,2.35
4,Atlanta Hawks,82.0,103.4,38.2,85.5,44.6,11.2,31.0,36.0,15.8,...,15.0,19.6,17-18,False,True,411660,5951,69224606.0,0.176806,3.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,Washington Wizards,82.0,109.2,41.3,87.0,47.5,9.2,24.8,37.2,17.3,...,13.5,21.3,16-17,True,False,525176,6218,97430036.0,0.153843,1.75
235,Washington Wizards,82.0,106.6,39.9,85.6,46.7,9.9,26.5,37.5,16.8,...,14.0,21.3,17-18,True,True,547123,6256,123055869.0,0.161488,3.05
236,Washington Wizards,82.0,114.0,42.1,90.1,46.8,11.3,33.3,34.1,18.4,...,13.7,20.7,18-19,False,True,566892,6297,115658099.0,0.150464,3.05
237,Washington Wizards,72.0,114.4,41.5,90.9,45.7,12.0,32.6,36.8,19.4,...,13.6,22.7,19-20,False,False,561028,6386,85196283.0,0.159513,3.35


## h. Offensive Index= （Avg Points per game- Min PTS）/（Max PTS-Min PTS）+ 3P% + normalized Offensive Rebound+ normalized Assistance

In [352]:
final_df[final_df.year=='15-16']['3P%'].max()

41.6

In [353]:
annual_min_stats = final_df.groupby('year').min()[['PTS','3P%','REB','AST']].reset_index()

In [354]:
annual_max_stats = final_df.groupby('year').max()[['PTS','3P%','REB','AST']].reset_index()

In [355]:
annual_min_stats.columns = ['year','PTS_min','3P%_min','REB_min','AST_min']

In [356]:
annual_max_stats.columns = ['year','PTS_max','3P%_max','REB_max','AST_max']

In [357]:
wip = final_df.merge(annual_max_stats,on='year').merge(annual_min_stats,on='year')

In [358]:
wip

,TEAM,GP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,...,turnover_metrics,median_ast,PTS_max,3P%_max,REB_max,AST_max,PTS_min,3P%_min,REB_min,AST_min
0,Atlanta Hawks,82.0,101.0,37.3,81.6,45.8,9.4,25.8,36.3,17.0,...,0.168249,3.00,107.9,39.7,46.4,25.2,93.7,31.2,36.9,18.9
1,Boston Celtics,82.0,96.2,36.5,83.9,43.5,7.0,21.1,33.3,16.2,...,0.170993,2.60,107.9,39.7,46.4,25.2,93.7,31.2,36.9,18.9
2,Brooklyn Nets,82.0,98.5,35.7,77.9,45.9,8.6,23.4,36.9,18.4,...,0.155597,2.20,107.9,39.7,46.4,25.2,93.7,31.2,36.9,18.9
3,Chicago Bulls,82.0,93.7,34.7,80.2,43.2,6.2,17.8,34.8,18.1,...,0.167743,3.40,107.9,39.7,46.4,25.2,93.7,31.2,36.9,18.9
4,Cleveland Cavaliers,82.0,98.2,37.0,84.8,43.7,7.1,20.0,35.6,17.0,...,0.165841,2.95,107.9,39.7,46.4,25.2,93.7,31.2,36.9,18.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,Sacramento Kings,72.0,113.7,42.6,88.6,48.1,12.1,33.3,36.4,16.4,...,0.148753,3.55,120.1,41.1,48.3,27.7,103.8,33.6,41.4,21.3
235,San Antonio Spurs,72.0,111.1,41.9,90.5,46.2,9.9,28.4,35.0,17.4,...,0.124106,2.15,120.1,41.1,48.3,27.7,103.8,33.6,41.4,21.3
236,Toronto Raptors,72.0,111.3,39.7,88.7,44.8,14.5,39.3,36.8,17.4,...,0.122274,2.70,120.1,41.1,48.3,27.7,103.8,33.6,41.4,21.3
237,Utah Jazz,72.0,116.4,41.3,88.1,46.8,16.7,43.0,38.9,17.2,...,0.161128,2.50,120.1,41.1,48.3,27.7,103.8,33.6,41.4,21.3


In [359]:
wip['normalized_pts'] = (wip.PTS-wip.PTS_min)/(wip.PTS_max-wip.PTS_min)
wip['normalized_3p%'] = (wip['3P%']-wip['3P%_min'])/(wip['3P%_max']-wip['3P%_min'])
wip['normalized_reb'] = (wip.REB-wip.REB_min)/(wip.REB_max-wip.REB_min)
wip['normalized_ast'] = (wip.AST-wip.AST_min)/(wip.AST_max-wip.AST_min)

In [360]:
wip['offensive_index'] = wip.normalized_ast + wip.normalized_pts + wip.normalized_reb + wip['normalized_3p%']

In [361]:
final_df = final_df.merge(wip[['year','TEAM','offensive_index']], on=['year','TEAM'])

In [362]:
len(final_df.TEAM.unique())


30

## Modeling

### Logistic Regression

In [363]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [364]:
final_df.columns


Index(['TEAM', 'GP', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM',
       'FTA', 'FT%', 'OR', 'DR', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF',
       'year', 'playoff', 'lasy_year_playoff', 'GDP', 'pop', 'SALARY',
       'turnover_metrics', 'median_ast', 'offensive_index'],
      dtype='object')

In [365]:
final_df =final_df.reset_index()

In [366]:
training_x = final_df[final_df.year<'19-20'][ ['lasy_year_playoff', 'GDP', 'pop', 'SALARY',
       'turnover_metrics', 'median_ast', 'offensive_index']]
training_y = final_df[final_df.year<'19-20']['playoff']

In [367]:
testing_x = final_df[final_df.year>='19-20'][ ['lasy_year_playoff', 'GDP', 'pop', 'SALARY',
       'turnover_metrics', 'median_ast', 'offensive_index']]
testing_y = final_df[final_df.year>='19-20'][['playoff','index']]

In [368]:
model = LogisticRegression()

In [369]:
import statsmodels.formula.api as smf

In [370]:
training_x.dtypes

lasy_year_playoff       bool
GDP                    int64
pop                    int64
SALARY               float64
turnover_metrics     float64
median_ast           float64
offensive_index      float64
dtype: object

In [371]:
training_y[training_y.astype(float)<0]

Series([], Name: playoff, dtype: bool)

In [372]:
stats_data =final_df[final_df.year>='19-20'][ ['lasy_year_playoff', 'GDP', 'pop', 'SALARY',
       'turnover_metrics', 'median_ast', 'offensive_index','playoff']]

In [373]:
stats_data.playoff= stats_data.playoff.astype(int)
stats_data.lasy_year_playoff= stats_data.lasy_year_playoff.astype(int)

In [374]:
log_reg = smf.logit(formula='playoff ~ GDP + lasy_year_playoff + pop + SALARY + turnover_metrics + median_ast + offensive_index',data=stats_data).fit()

Optimization terminated successfully.
         Current function value: 0.296392
         Iterations 8


In [375]:
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                playoff   No. Observations:                   60
Model:                          Logit   Df Residuals:                       52
Method:                           MLE   Df Model:                            7
Date:                Fri, 22 Apr 2022   Pseudo R-squ.:                  0.5710
Time:                        13:45:40   Log-Likelihood:                -17.784
converged:                       True   LL-Null:                       -41.455
Covariance Type:            nonrobust   LLR p-value:                 4.783e-08
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -4.9104      4.517     -1.087      0.277     -13.763       3.942
GDP                2.917e-06   5.02e-06      0.581      0.562   -6.93e-06    1.28e-05
lasy_year_playoff     2.6794      1.083      2.474      0.013       0.557       4.802
pop                 3.35e-05      0.000      0.074      0.941      -0.001       0.001
SALARY              6.87e-08   3.31e-08      2.075      0.038    3.82e-09    1.34e-07
turnover_metrics    -45.8821     25.255     -1.817      0.069     -95.381       3.617
median_ast           -0.6011      0.531     -1.133      0.257      -1.641       0.439
offensive_index       1.9355      0.915      2.115      0.034       0.142       3.729
=====================================================================================
"""

In [376]:
testing_x.lasy_year_playoff= testing_x.lasy_year_playoff.astype(int)
results_log_reg = final_df[final_df.year>='19-20'][['year','TEAM','lasy_year_playoff', 'GDP', 'pop', 'SALARY',
       'turnover_metrics', 'median_ast', 'offensive_index','playoff']]\
.join(pd.DataFrame({'prediction':log_reg.predict(testing_x)}))

In [377]:
playoffs_testing = results_log_reg.sort_values(['year','prediction'],ascending=False).groupby(['year']).head(14)

In [378]:
playoffs_testing[['predicted_result']] = 1

In [379]:
results_log_reg = results_log_reg.join(playoffs_testing.predicted_result)
results_log_reg.predicted_result = results_log_reg.predicted_result.fillna(0)
results_log_reg.playoff = results_log_reg.playoff.astype(int)

In [380]:
len(results_log_reg.predicted_result)

60

In [381]:
results_log_reg.sort_values(['year','prediction'],ascending=False).groupby(['year']).head(3)

,year,TEAM,lasy_year_playoff,GDP,pop,SALARY,turnover_metrics,median_ast,offensive_index,playoff,prediction,predicted_result
23,20-21,Brooklyn Nets,True,1912455,19768,180381490.0,0.139303,4.70,2.948800,1,0.999997,1.0
102,20-21,Los Angeles Clippers,True,1064438,13036,131781660.0,0.142007,3.25,2.515939,1,0.998847,1.0
134,20-21,Milwaukee Bucks,True,92963,1566,123122306.0,0.147120,3.20,3.333931,1,0.987011,1.0
101,19-20,Los Angeles Clippers,True,1007037,13110,122699675.0,0.154540,3.80,2.726309,1,0.995836,1.0
85,19-20,Houston Rockets,True,488165,7154,137346433.0,0.146086,1.55,1.609872,1,0.987256,1.0
197,19-20,Portland Trail Blazers,True,168380,2516,126324546.0,0.129466,1.60,2.061993,1,0.983892,1.0


In [382]:
from sklearn.metrics import (confusion_matrix,
                           accuracy_score)
 
cm = confusion_matrix(results_log_reg.playoff, results_log_reg.predicted_result)
print ("Confusion Matrix : \n", cm)
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(results_log_reg.playoff, results_log_reg.predicted_result))

Confusion Matrix : 
 [[25  3]
 [ 7 25]]
Test accuracy =  0.8333333333333334


## Random Forest Classification 

In [383]:
from sklearn.ensemble import RandomForestClassifier

In [384]:
def randomForestClassifier(x_train,x_test, y_train):
    rf_clf = RandomForestClassifier(random_state=614)
    rf_clf.fit(x_train, y_train)

    y_predict_train = rf_clf.predict(x_train)
    y_predict_test = rf_clf.predict(x_test)
    # -------------------------------
    return rf_clf,y_predict_train, y_predict_test

In [385]:
training_x.lasy_year_playoff= training_x.lasy_year_playoff.astype(int)
training_y= training_y.astype(int)

In [386]:
rfm, y_predict_train, predictions_rf = randomForestClassifier(training_x,testing_x, training_y)

In [387]:
predictions_rf

array([0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1])

In [388]:
np.argsort(rfm.feature_importances_)[::-1]

array([6, 3, 5, 1, 4, 2, 0], dtype=int64)

In [389]:
{i[0]:i[1]for i in zip(training_x.columns.tolist(),rfm.feature_importances_.tolist())}

{'lasy_year_playoff': 0.0726632097679087,
 'GDP': 0.13092935938476635,
 'pop': 0.10964234569805731,
 'SALARY': 0.1768051915463171,
 'turnover_metrics': 0.11848677267363206,
 'median_ast': 0.1551595757536755,
 'offensive_index': 0.23631354517564307}

In [390]:
from sklearn.metrics import accuracy_score, classification_report

In [391]:
accuracy_score(testing_y.playoff.astype(int),predictions_rf)

0.7333333333333333

In [392]:
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
def hyperParameterTuning(rf_clf,x_train,y_train):
    params = {
        'n_estimators': [4, 16, 256],
        'max_depth': [2, 8, 16]
    }
    gscv_rfc = GridSearchCV(estimator=rf_clf, param_grid=params)
    gscv_rfc_fit = gscv_rfc.fit(x_train, y_train)
    # -------------------------------
    return gscv_rfc, gscv_rfc_fit

In [393]:
tuned_rfm_search, tuned_rfm = hyperParameterTuning(rfm,training_x,training_y)

In [394]:
predictions_tuned_rfm = tuned_rfm.predict(testing_x)
accuracy_score(testing_y.playoff.astype(int),predictions_tuned_rfm)

0.7166666666666667

In [395]:
rfm_predictions = pd.DataFrame({'rfm_prediction':tuned_rfm.predict_proba(testing_x)[:,1]},index=testing_y.index)

In [396]:
results_rfm = final_df[final_df.year>='19-20'][['year','TEAM','lasy_year_playoff', 'GDP', 'pop', 'SALARY',
       'turnover_metrics', 'median_ast', 'offensive_index','playoff']].join(rfm_predictions)

In [397]:
rfm_playoffs = results_rfm.sort_values(['year','rfm_prediction'],ascending=False).groupby(['year']).head(14)
rfm_playoffs['rfm_predicted_result'] = 1
results_rfm = results_rfm.join(rfm_playoffs.rfm_predicted_result)
results_rfm.rfm_predicted_result = results_rfm.rfm_predicted_result.fillna(0)
results_rfm.playoff = results_rfm.playoff.astype(int)

In [398]:
from sklearn.metrics import (confusion_matrix,
                           accuracy_score)
 
cm = confusion_matrix(results_rfm.playoff, results_rfm.rfm_predicted_result)
print ("Confusion Matrix : \n", cm)
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(results_rfm.playoff, results_rfm.rfm_predicted_result))

Confusion Matrix : 
 [[21  7]
 [11 21]]
Test accuracy =  0.7


In [402]:
results_rfm.sort_values(['year','rfm_prediction'],ascending=False).groupby(['year']).head(1)

,year,TEAM,lasy_year_playoff,GDP,pop,SALARY,turnover_metrics,median_ast,offensive_index,playoff,rfm_prediction,rfm_predicted_result
230,20-21,Utah Jazz,True,109836,1263,128955106.0,0.161128,2.5,2.854673,1,0.759676,1.0
221,19-20,Toronto Raptors,True,328359,6197,118082968.0,0.166193,2.7,2.565892,1,0.770539,1.0


In [400]:
len(results_log_reg)

60

In [411]:
final_results = log_reg.predict(final_df[['lasy_year_playoff', 'GDP', 'pop', 'SALARY',
       'turnover_metrics', 'median_ast', 'offensive_index']].astype(float))

final_results_df = final_df[['year','TEAM','lasy_year_playoff', 'GDP', 'pop', 'SALARY',
       'turnover_metrics', 'median_ast', 'offensive_index','playoff']]\
.join(pd.DataFrame({'prediction':final_results}))
final_results_df

,year,TEAM,lasy_year_playoff,GDP,pop,SALARY,turnover_metrics,median_ast,offensive_index,playoff,prediction
0,13-14,Atlanta Hawks,True,325003,5595,52150108.0,0.168249,3.00,2.392781,True,0.082840
1,14-15,Atlanta Hawks,True,347745,5689,52112642.0,0.168001,2.95,2.315523,True,0.079763
2,15-16,Atlanta Hawks,True,369996,5792,68126061.0,0.172936,3.25,1.464703,True,0.034536
3,16-17,Atlanta Hawks,True,389472,5877,88612062.0,0.167276,2.35,1.654266,True,0.332694
4,17-18,Atlanta Hawks,True,411660,5951,69224606.0,0.176806,3.05,1.473953,False,0.040398
...,...,...,...,...,...,...,...,...,...,...,...
234,16-17,Washington Wizards,False,525176,6218,97430036.0,0.153843,1.75,2.322184,True,0.476841
235,17-18,Washington Wizards,True,547123,6256,123055869.0,0.161488,3.05,2.265754,True,0.959734
236,18-19,Washington Wizards,True,566892,6297,115658099.0,0.150464,3.05,1.791375,False,0.909670
237,19-20,Washington Wizards,False,561028,6386,85196283.0,0.159513,3.35,2.169196,False,0.087789
